# RAG sobre Historia y Cultura de Uruguay

### Autores
- Steven Estramil
- Daniel Padron
- Agustin Recoba

Este proyecto implementa un sistema de Recuperación Aumentada por Generación (RAG) para responder preguntas sobre la historia, cultura y demografía de Uruguay, utilizando documentos extraídos de Wikipedia y modelos de lenguaje de última generación.

## 1. Adquisición y Procesamiento de Documentos

Se seleccionaron artículos relevantes de Wikipedia en español sobre Uruguay, abarcando historia, cultura, demografía y figuras clave. Los textos se extraen y almacenan como cadenas de texto para su posterior procesamiento.

**Fuentes:** Wikipedia, utilizando la librería [wikipedia-api](https://github.com/martin-majlis/Wikipedia-API/).

---

El dominio seleccionado es la historia y cultura de Uruguay, incluyendo su proceso de independencia, demografía y los pueblos originarios. Los documentos se obtienen directamente de Wikipedia para asegurar calidad y actualidad de la información.

In [ ]:
nombres_paginas_wikipedia = [
    "Historia_de_Uruguay",
    "Charrúas",
    "Cultura_de_Uruguay",
    "Demografía_de_Uruguay",
    "José_Gervasio_Artigas",
    "Batalla_de_Las_Piedras_(1811)",
    "Declaratoria_de_la_independencia_(Uruguay)",
    "Revolución_oriental",
]

In [ ]:
# Descargando contenido de wikipedia

import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia("PROYECTO_RAG", "es")

contenido_wiki = [
    wiki_wiki.page(nombre_web).text for nombre_web in nombres_paginas_wikipedia
]

Imprimimos el primer sitio(documento).

In [ ]:
contenido_wiki[0]

'La historia de Uruguay abarca la historia del actual territorio de Uruguay, desde antes de que se constituyera como estado hasta la actualidad.\n\nÉpoca indígena\nExisten indicios de población en la Banda Oriental desde varios miles de años atrás,[1]\u200b como lo testimonian los hallazgos de la Cultura Catalanense en Artigas, datados de aproximadamente 10  000 años  a.  C., lo que la convierte en el yacimiento arqueológico humano más antiguo del país. También destacan los cerritos de indios al este del territorio, con una antigüedad de 5000 años; y los recientes hallazgos en Mataojo (Salto).[2]\u200b Otro hallazgo significativo son los antropolitos, zoolitos y ornitolitos.[3]\u200b También es revelador el yacimiento del Vizcaíno, cerca de Sauce, rico en megafauna con marcas de herramientas líticas, lo cual indica la presencia humana hace varios miles de años en el lugar.[4]\u200b\nLos habitantes conocidos del actual territorio de Uruguay antes de la llegada de los españoles eran las 

Se decide no realizar en esta etapa ningún filtrado sobre el documento, ya que no se identifican secciones que tengan un efecto negativo en el sistema. Tanto los títulos, enumerados y pies de página pueden contener información de utilidad para el sistema de recuperación. Aunque otros elementos dentro de los documentos, como las URLs podrían ser candidatas a ser suprimidas en trabajos futuros.

## 2. Segmentación de Documentos (_Chunking_)

Para optimizar la recuperación semántica, los documentos se dividen en segmentos (_chunks_) de hasta 500 caracteres, priorizando la preservación de la estructura textual (párrafos, oraciones, palabras). Se emplea el `RecursiveCharacterTextSplitter` de LangChain, ajustando los separadores para maximizar el contexto de cada fragmento.

Se eliminan los _chunks_ menores a 35 caracteres para evitar respuestas basadas en información insuficiente.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False,
    separators=[
        "\n\n",
        "\n",
        ".",
        ",",
        " ",
        "",
    ],
)


def chunk_text(text):
    chunks_obtenidos = text_splitter.split_text(text)

    return [re.sub(r"[\u200b]", "", chunk) for chunk in chunks_obtenidos]

In [ ]:
chunks = []
for document in contenido_wiki:
    chunks += chunk_text(document)

Se analizan los diferentes chunks obtenidos, tanto por cantidad, como por su largo.

In [ ]:
canrtidad_chunks = len(chunks)
print(f"Cantidad de chunks: {canrtidad_chunks}")
largo_medio = sum([len(chunk) for chunk in chunks]) / len(chunks)
print(f"Largo medio de los chunks: {largo_medio:.1f}")
largo_minimo = min([len(chunk) for chunk in chunks])
print(f"Largo minimo de los chunks: {largo_minimo}")
largo_maximo = max([len(chunk) for chunk in chunks])
print(f"Largo maximo de los chunks: {largo_maximo}")
cant_pocos_caracteres = len([chunk for chunk in chunks if len(chunk) < 150])
print(f"Cantidad de chunks con menos de 150 caracteres: {cant_pocos_caracteres}")
cant_muy_pocos_caracteres = len([chunk for chunk in chunks if len(chunk) < 35])
print(f"Cantidad de chunks con menos de 35 caracteres: {cant_muy_pocos_caracteres}")

Cantidad de chunks: 1284
Largo medio de los chunks: 312.9
Largo minimo de los chunks: 1
Largo maximo de los chunks: 499
Cantidad de chunks con menos de 150 caracteres: 171
Cantidad de chunks con menos de 35 caracteres: 60


Dado que el número de chunks retornados por el sistema de recuperación no depende de la longitud de los chunks, **se decide eliminar los chunks cuya longitud sea menor a 30 caracteres**. De esta forma se busca evitar que el LLM base sus respuestas en chunks de pequeña longitud, y en consecuencia limitar sus posibilidades de responder correctamente.

In [ ]:
chunks = list(filter(lambda chunk: len(chunk) > 35, chunks))

## 3. Recuperación Semántica de Información

Se implementa un sistema de recuperación basado en modelos Bi-Encoder multilingües, utilizando `SentenceTransformer` y `NearestNeighbors` para obtener los _chunks_ más relevantes ante una consulta. Se experimenta con distintos modelos de embeddings y estrategias de recuperación para comparar su impacto en la calidad de las respuestas.

In [ ]:
from sentence_transformers import SentenceTransformer

A continuación se debe generar las representaciones vectoriales para todos los _chunks_ ([ejemplo de uso](https://huggingface.co/intfloat/multilingual-e5-large#support-for-sentence-transformers)).

**Observación:** El modelo que estamos usando espera que los _chunks_ comiencen con el prefijo `passage: ` por lo que será necesario agregarlo al inicio de todos los _chunks_.

Por último, se debe implementar el algoritmo de búsqueda de los embeddings más cercanos para un embedding dado.

**Sugerencias:**
* Utilizar la clase NearestNeighbors de sklearn ([documentación](https://scikit-learn.org/dev/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)).

---
Con el fin de probar diferentes encoders y modelos de lenguaje, se crean el par de clases abstractas:  
- **ChunkRetreiver**: Engloba el comportamiento referido a la obtención de chunks para una consulta determinada. Contiene la firma del método utilizado para obtener el número de Chunk más cercanos, `get_n_closest_chunks`
- **RAGenerator**: Engloba el comportamiento del RAG (consulta el ChunkRetreiver por los chunk), y luego vía el LLM genera la sentencia con `get_answer`



In [ ]:
from abc import ABC, abstractmethod


class ChunkRetreiver(ABC):
    def __init__(self, chunks: list):
        self.chunks = chunks

    @abstractmethod
    def get_n_closest_chunks(self, query: str, n=3):
        pass


class RAGenerator(ABC):
    def __init__(self, chunk_retreiver: ChunkRetreiver):
        self.chunk_retreiver = chunk_retreiver

    @abstractmethod
    def get_answer(self, *args, **kwargs):
        pass

    def __call__(self, *args, **kwargs):
        return self.get_answer(*args, **kwargs)

A continuación se crea la implementación más inmediata, y que se utilizara como línea base, la clase `RandomChunkRetreiver` el cual retorna aleatoriamente n chunks a utilizar en la consulta.

In [ ]:
from random import choices


class RandomChunkRetreiver(ChunkRetreiver):
    def get_n_closest_chunks(self, query: str, n=3):
        return choices(self.chunks, k=n)


randomretriver = RandomChunkRetreiver(chunks)

randomretriver.get_n_closest_chunks("Cuando fue el éxodo del pueblo oriental?")

['. Le tocaría al Cabildo decidir en cada caso si esta tropa se integraría con vecinos “que deberán mudarse mensualmente” o con soldados profesionales.',
 'Diversos incidentes llevaron a la revolución de Rivera y sus partidarios en contra del gobierno en 1836. Oribe decretó que sus seguidores usaran una divisa blanca, mientras Rivera adoptó una cinta colorada (primero había elegido una celeste, pero debido a que se confundía con el blanco decidió cambiarla). Es de esta forma que nacieron los «blancos» y «colorados», quienes se enfrentaron por primera vez en la batalla de Carpintería, el 19 de septiembre de ese año.',
 'El militarismo significó desde el ángulo político, la sustitución de las banderías tradicionales (blancos y colorados), por el gobierno de los grupos de presión más fuertes en lo económico, aliados al grupo de presión más fuerte en el poder real y coactivo: el ejército']

Con el fin de experimentar con varios encoders multilingües, se llevó a cabo una relevación en Hugging Face.
Primeramente, fue de interés experimentar con encoders de tipo cross-encoder y colBert, pero no fue posible en multilingue. Para el caso particular del **cross-encoder**, se buscó utilizar los modelos vía la clase [CrossEncoder](\https://www.sbert.net/examples/applications/cross-encoder/README.html), pero solamente fue posible con el modelo `ms-marco-MiniLM-L-2-v2`, cuyos resultados fueron deficientes.
El resto de los modelos dentro de [leaderboard](https://huggingface.co/spaces/mteb/leaderboard) no tenían parámetros (pesos) disponibles para el clasificador final del cross-encoder.

En base a esto último, se incluye en la entrega un total de 3 encoders, todos de tipo bi-encoder. Los dos primeros `multilingual-e5-large` y `KaLM-embedding-multilingual-mini-v1` presentan resultados destacados en tareas de recuperación y son multilingües.
Por último, se experimenta con la versión **instruct** del primero, `multilingual-e5-large-instruct`, el cual acepta adicionalmente una tarea. Este último modelo reporta un mejor desempeño que su variante básica (adelanto, en la tarea del actual laboratorio también).
La tarea utilizada en el modelo instruct fue: "*Dada una pregunta, retorna los documentos más relevantes que la responden totalmente o parcialmente.*".

Se procede con una segunda implementacion de `ChunkRetreiver`. La clase es de carácter genérico, y vía el uso de SentenceTransformer, es capaz de hacer uso de una amplia gama de bi-encoders.
Se comienzan calculando los embeddings de todos los chunks. Luego, cada vez que se requiera calcular los `n` chunks más cercanos, se realizara la búsqueda de los chunks via `NearestNeighbors`

In [ ]:
from sklearn.neighbors import NearestNeighbors


class BiEncoderChunkRetreiver(ChunkRetreiver):
    def __init__(self, chunks: list, model_emb: SentenceTransformer):
        super().__init__(chunks)
        self.model_emb = model_emb

        self.embedings_chunks = model_emb.encode(
            ["passage: " + chunk for chunk in chunks]
        )

        self.n_neignbors = NearestNeighbors(n_neighbors=5, metric="cosine")
        self.n_neignbors = self.n_neignbors.fit(model_emb.encode(chunks))

    def get_n_closest_chunks(self, query, n=3):
        emb = self.model_emb.encode(["query: " + query])

        indices = list(self.n_neignbors.kneighbors(emb, return_distance=False))[0]

        return [self.chunks[i] for i in indices[0:n]]


In [ ]:
class BiEncoderInstructChunkRetreiver(ChunkRetreiver):
    def __init__(self, chunks: list, model_emb: SentenceTransformer):
        super().__init__(chunks)
        self.model_emb = model_emb

        self.task_prompt = "Dada una pregunta, retorna los documentos más relevantes que la responden totalmente o parcialmente"

        self.embedings_chunks = model_emb.encode(
            ["passage: " + chunk for chunk in chunks]
        )

        self.n_neignbors = NearestNeighbors(n_neighbors=5, metric="cosine")
        self.n_neignbors = self.n_neignbors.fit(model_emb.encode(chunks))

    def get_n_closest_chunks(self, query, n=3):
        emb = self.model_emb.encode(
            ["Instruct:" + self.task_prompt + "\nQuery:" + query]
        )

        indices = list(self.n_neignbors.kneighbors(emb, return_distance=False))[0]

        return [self.chunks[i] for i in indices[0:n]]

Descargamos los modelos y comenzamos a instanciar los sistemas de recuperación (ChunkRetreiver). En todos los casos se realiza una primera prueba, solicita al sistema de recuperación los chunks más cercanos ante la pregunta "*¿Cuando fue el éxodo del pueblo oriental?*"

**Atención, dado el numero de modelos utilizados, en caso de correr esta celda no se recomienda hacerlo sin haber eliminado primero de memoria otros modelos.**


Se instancian las clases relacionadas al bi-encoder `multilingual-e5-large`

In [ ]:
model_emb_multilingual_e5_large = SentenceTransformer("intfloat/multilingual-e5-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
biencoder_multilingual_e5_large = BiEncoderChunkRetreiver(
    chunks, model_emb_multilingual_e5_large
)

biencoder_multilingual_e5_large.get_n_closest_chunks(
    "Cuando fue el éxodo del pueblo oriental?"
)

['Artigas, disgustado por el armisticio y ante la evacuación de las tropas porteñas, cumplió con su nuevo cargo trasladándose al territorio misionero, por lo que decidió pasar con sus seguidores a la orilla occidental del río Uruguay, hecho conocido como el éxodo oriental',
 'Artigas acató el pacto, retirando el sitio de Montevideo. Casi toda la población del sur de la Banda Oriental excepto Montevideo, lo acompañó en la retirada hasta el arroyo Ayuí Grande en la actual ciudad entrerriana de Concordia, dejando el territorio virtualmente vacío. A este acontecimiento se le denomina el Éxodo oriental o también La Redota y demostró el liderazgo o rol de caudillo de José Gervasio Artigas sobre el pueblo de la Banda Oriental.',
 'Sin embargo, en la lucha por resistir las imposiciones del gobierno de Buenos Aires y la consiguiente aspiración a lograr su autonomía política, fue gestándose un sentimiento particularista y nacionalista (denominado la «orientalidad» que tiene sus raíces en el Éxod

Se instancian las clases relacionadas al bi-encoder `KaLM-embedding-multilingual-mini-v1`

In [ ]:
model_emb_KaLM_multilingual = SentenceTransformer(
    "HIT-TMG/KaLM-embedding-multilingual-mini-v1"
)

In [ ]:
biencoder_model_emb_KaLM_multilingual = BiEncoderChunkRetreiver(
    chunks, model_emb_KaLM_multilingual
)

biencoder_model_emb_KaLM_multilingual.get_n_closest_chunks(
    "Cuando fue el éxodo del pueblo oriental?"
)

['El Éxodo Oriental\nEn ese clima de frustración y derrota quedó de manifiesto la voluntad de los orientales de reanudar el combate apenas las circunstancias lo permitieran. De inmediato Artigas al frente de 3.000 soldados retomó su camino hacia en Norte, y un alto número de civiles lo acompañó. El caudillo se opuso a esta emigración masiva en un principio, pero luego ordenó levantar un registro de las familias e individuos que lo seguían.',
 'El éxodo del pueblo oriental\nComo consecuencia del armisticio firmado con el virrey Francisco Javier de Elío por la Primera Junta de Buenos Aires, las tropas enviadas a la Banda Oriental debieron abandonar dicho territorio, levantando el sitio de Montevideo.\nJosé Artigas fue nombrado «teniente gobernador, justicia mayor y capitán del departamento de Yapeyú», entonces en la provincia de Misiones, actual Argentina.',
 'Artigas acató el pacto, retirando el sitio de Montevideo. Casi toda la población del sur de la Banda Oriental excepto Montevideo,

Se instancian las clases relacionadas al bi-encoder `multilingual-e5-large-instruct`   

In [ ]:
model_emb_multilingual_e5_large_instruct = SentenceTransformer(
    "intfloat/multilingual-e5-large-instruct"
)

In [ ]:
biencoder_multilingual_e5_large_instruct = BiEncoderInstructChunkRetreiver(
    chunks, model_emb_multilingual_e5_large_instruct
)

biencoder_multilingual_e5_large_instruct.get_n_closest_chunks(
    "Cuando fue el éxodo del pueblo oriental?"
)

['La historia de Uruguay abarca la historia del actual territorio de Uruguay, desde antes de que se constituyera como estado hasta la actualidad.',
 'La Suiza de América\nA finales del siglo XIX el país había completado su organización y durante la etapa batllista consolidó su democracia y alcanzó altos niveles de bienestar, equiparables a los europeos. Debido a esto, Uruguay comenzó a ser conocido como «la Suiza de América».',
 'Origen y descendencia\nUruguay es una sociedad multiétnica,[8] lo cual significa que históricamente ha estado habitada por personas de diferentes orígenes étnicos. Por ello los uruguayos generalmente tratan su nacionalidad como una ciudadanía genérica, en lugar de enfatizar en un determinado grupo étnico o en un determinado origen cultural.']

## 4. Generación de Respuestas con LLMs

Se integran modelos de lenguaje de última generación (Llama 3.1 y Qwen2.5) a través de HuggingFace para la generación de respuestas. El sistema construye prompts que incluyen los _chunks_ recuperados y, opcionalmente, ejemplos de few-shot para mejorar la precisión y el estilo de las respuestas.

La arquitectura permite experimentar con diferentes modelos y configuraciones de prompting.

### Configuración de LLM

Utilizaremos el modelo **Llama 3.1** de Meta a través de la plataforma [HuggingFace](https://huggingface.co/). Para poder usar este modelo en HuggingFace es necesario seguir los siguientes pasos:

- Crearse una cuenta de HuggingFace (https://huggingface.co/)
- Aceptar los términos para usar el modelo en HuggingFace, que aparecen en el siguiente enlace: https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct
- Crear un token de HuggingFace con permiso de lectura siguiendo el siguiente enlace: https://huggingface.co/settings/tokens
- Ejecutar la siguiente celda e ingresar el token creado.

In [ ]:
# Ejecutar para conectarse a HuggingFace
from huggingface_hub import notebook_login

notebook_login()

A continuación se define la inicialización del tokenizer y del modelo cuantizado a 4 bits.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch


def get_model_and_tokenizer(name_model):
    # Inicializar el tokenizer
    tokenizer = AutoTokenizer.from_pretrained(name_model)

    # Configuración de cuantización a 4 bits (para mejorar eficiencia)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    # Inicializar el modelo
    model = AutoModelForCausalLM.from_pretrained(
        name_model,
        quantization_config=bnb_config,
        device_map="auto",
    )
    return model, tokenizer

### Crear prompt y generar respuesta

Escribir la función `create_prompt(question)` que dada una pregunta, genere la prompt que se utilizará para generar la respuesta. Tener en cuenta que se debe realizar la búsqueda semántica de los _chunks_ más cercanos a la pregunta utilizando lo implementado en la parte 3.

**Observación:** Al igual que para los _chunks_, el modelo Bi-Encoder espera que la pregunta comience con un prefijo especial: `query: ` por lo que será necesario agregarlo al inicio de la pregunta para generar el embedding.

**Sugerencias:**
* Puede probar con distintas cantidades de _chunks_ recuperados, pero se sugiere comenzar con 3. Tener en cuenta que más _chunks_ recuperados y agregados en la prompt implica mayor uso de memoria en inferencia.
* Utilizar la función `apply_chat_template` del tokenizer para aplicar el template correcto del modelo Llama 3.1.

---
Se crea la clase `ModelGenerator`, heredada de la clase `RAGenerator`. La misma implementa los métodos asociados a la generación de la respuesta, siendo el primero de estos `get_response`, enviando el prompt junto con sus configuraciones al LLM. Seguido el método `create_prompt` que construye el prompt haciendo uso de la función `apply_chat_template`; notar que se utiliza el rol `system` para la definición de la tarea y el envío de los chunks, el rol `assistant` para simular respuestas del modelo para el caso de few-shot, y por último `user` para el envío de las preguntas, tanto en los ejemplos del few-shot como la pregunta de entrada del RAG.

In [ ]:
# Generar respuesta
from transformers import GenerationConfig, pipeline


class ModelGenerator(RAGenerator):
    def get_answer(
        self,
        model_LLM,
        tokenizer,
        question,
        temp=0.0,
        max_tok=500,
        num_chunk_retrieve=3,
        pprint=False,
        few_shots_examples=[],
    ):
        prompt = self.create_prompt(
            tokenizer, question, num_chunk_retrieve, few_shots_examples
        )
        if pprint:
            print("PROMPT:")
            print(prompt)

        r = self.get_response(model_LLM, tokenizer, prompt, temp, max_tok).split(
            "assistant\n\n"
        )[-1]
        if pprint:
            print("\nRESPUESTA:")
            print(r)

        return r

    def get_response(self, model_LLM, tokenizer, prompt, temp=0.0, max_tok=500):
        # Configuración de temperatura
        generation_config = GenerationConfig(
            temperature=temp if temp > 0 else None, do_sample=temp > 0
        )

        # Inicializar pipeline para generación de texto
        pipe = pipeline(
            "text-generation",
            model=model_LLM,
            config=generation_config,
            tokenizer=tokenizer,
            pad_token_id=tokenizer.eos_token_id,
        )

        # Generar texto
        output = pipe(prompt, return_full_text=False, max_new_tokens=max_tok)

        return output[0]["generated_text"]

    def create_prompt(
        self, tokenizer, question, num_chunk_retrieve, few_shots_examples=[]
    ):
        chunks_mas_cercanos = self.chunk_retreiver.get_n_closest_chunks(
            question, num_chunk_retrieve
        )

        messages = []

        messages.append(
            {
                "role": "system",
                "content": "Eres un sistema de preguntas y respuestas. Dar respuestas **CONCRETAS** y **PRECISAS**. Utiliza **ÚNICAMENTE** la información de los siguientes extractos para construir la respuesta:",
            }
        )

        for chunk in chunks_mas_cercanos:
            messages.append({"role": "system", "content": "extract: " + chunk})

        messages.append(
            {
                "role": "system",
                "content": "ES **MUY IMPORTANTE** que si los extractos no tienen una relación evidente con la pregunta, debes responder que no se cuenta con información para responder la pregunta.",
            }
        )
        for question_few_shot, response_few_shot in few_shots_examples:
            messages.append({"role": "user", "content": question_few_shot})
            messages.append({"role": "assistant", "content": response_few_shot})

        messages.append({"role": "user", "content": question})

        return tokenizer.apply_chat_template(messages, tokenize=False)


Con el fin de experimentar con la técnica de FewShot, a continuación se agregan 5 ejemplos adicionales. Estos, en caso de aplicar la técnica, serán enviados junto al prompt

In [ ]:
few_shots_examples = [
    (
        "¿En qué año Uruguay tenía una tasa de natalidad de 13,50 nacimientos por cada mil habitantes?",
        "En 2013 el país tenía una tasa de natalidad de 13,50 nacimientos por cada mil habitantes.",
    ),
    (
        "¿Quienes tomararon como prisionero al pirata inglés Francis Drake?",
        "En 1570, Francis navegaba en el río de la Plata y cayó prisionero de nativos charruas.",
    ),
    (
        "¿Qué tratado violaron los portugueses cuando invadieron la banda oriental en 1680?",
        "Los portugueses violaron el Tratado de Tordesillas.",
    ),
    (
        "¿Cómo se llamaba el primer gobernador de Montevideo?",
        "El primer gobernador se llamaba José Joaquín de Viana.",
    ),
    (
        "¿Quién y cuando fue aclamado como 'Primer Jefe de los Orientales?",
        "José Gervasio Artigas fue proclamado Primer Jefe de los Orientales el 10 de octubre de 1811.",
    ),
]

Se experimentarán con dos LLM muy conocidos: `Llama-3.1-8B-Instruct` y `Qwen2.5-7B-Instruct`. Para ello primero serán descargados.

**Atención, dado el numero de modelos utilizados, en caso de correr esta celda no se recomienda hacerlo sin haber eliminado primero de memoria otros modelos.**


In [ ]:
llama_model, llama_tokenizer = get_model_and_tokenizer(
    "meta-llama/Llama-3.1-8B-Instruct"
)

In [ ]:
qwen_model, qwen_tokenizer = get_model_and_tokenizer("Qwen/Qwen2.5-7B-Instruct")

Inicializamos los distintos sistemas RAG con los que se experimentaran en la próxima sección, variando entre estos el sistema de recuperación (uno de los 3 bi-encoders) y el de generación (uno de los dos LLM)

In [ ]:
qwen_biencoder_rag_model = ModelGenerator(biencoder_multilingual_e5_large)

In [ ]:
llama_biencoder_multilingual_e5_large_rag_model = ModelGenerator(
    biencoder_multilingual_e5_large
)
llama_biencoder_rag_KaLM_multilingual_rag_model = ModelGenerator(
    biencoder_model_emb_KaLM_multilingual
)
llama_biencoder_multilingual_e5_large_instruct_rag_model = ModelGenerator(
    biencoder_multilingual_e5_large_instruct
)
llama_random_rag_model = ModelGenerator(randomretriver)

### Ejemplo de Consulta

A continuación, se muestra un ejemplo de consulta al sistema RAG para la pregunta "¿Quién es Luis Lacalle Pou?". Se imprime el prompt generado y la respuesta del modelo.

In [ ]:
qwen_biencoder_rag_model(
    qwen_model, qwen_tokenizer, "Quien es Luis Lacalle Pou?", pprint=True
);

In [ ]:
llama_biencoder_multilingual_e5_large_rag_model(
    llama_model,
    llama_tokenizer,
    "Quien es Luis Lacalle Pou?",
    pprint=True,
    few_shots_examples=few_shots_examples,
);

PROMPT:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Eres un sistema de preguntas y respuestas. Dar respuestas **CONCRETAS** y **PRECISAS**. Utiliza **ÚNICAMENTE** la información de los siguientes extractos para construir la respuesta:<|eot_id|><|start_header_id|>system<|end_header_id|>

extract: Gobierno de Luis Lacalle Pou
Lacalle y la vicepresidenta Beatriz Argimón asumieron sus cargos el 1 de marzo de 2020. Tras el juramento constitucional ante la Asamblea General, desfilaron por avenida del Libertador en un Ford V8 descapotable de 1937, que perteneció a su bisabuelo Luis Alberto de Herrera. El recorrido finalizó en la Plaza Independencia, donde recibió del presidente saliente Tabaré Vázquez la banda presidencial.<|eot_id|><|start_header_id|>system<|end_header_id|>

extract: Lacalle había anunciado durante su campaña electoral la introducción de un paquete de medidas de gobierno a través de una ley de ur

In [ ]:
llama_biencoder_multilingual_e5_large_rag_model(
    llama_model,
    llama_tokenizer,
    "¿Artigas estaba Uruguay o Paraguay durante Declaratoria de la independencia en Uruguay? ",
    pprint=True,
    few_shots_examples=few_shots_examples,
);

PROMPT:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Eres un sistema de preguntas y respuestas. Dar respuestas **CONCRETAS** y **PRECISAS**. Utiliza **ÚNICAMENTE** la información de los siguientes extractos para construir la respuesta:<|eot_id|><|start_header_id|>system<|end_header_id|>

extract: José Gervasio Artigas (Montevideo, 19 de junio de 1764-Quinta Ybyray, Asunción del Paraguay, 23 de septiembre de 1850) fue un político, militar y estadista rioplatense que lideró la Revolución Oriental, actuó durante la Guerra de la Independencia de las Provincias Unidas del Río de la Plata y se destacó por ser el heraldo del federalismo en lo que hoy son Uruguay y Argentina.[2][3][4]Recibió los títulos de «Jefe de los Orientales»[5] y «Protector de los Pueblos Libres».<|eot_id|><|start_header_id|>system<|end_header_id|>

extract: La campaña de los Treinta y Tres Orientales inició la liberación de su provincia del I

In [ ]:
llama_biencoder_rag_KaLM_multilingual_rag_model(
    llama_model,
    llama_tokenizer,
    "Quien es Luis Lacalle Pou?",
    pprint=True,
    few_shots_examples=few_shots_examples,
);

PROMPT:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Eres un sistema de preguntas y respuestas. Dar respuestas **CONCRETAS** y **PRECISAS**. Utiliza **ÚNICAMENTE** la información de los siguientes extractos para construir la respuesta:<|eot_id|><|start_header_id|>system<|end_header_id|>

extract: Gobierno de Luis Lacalle Pou
Lacalle y la vicepresidenta Beatriz Argimón asumieron sus cargos el 1 de marzo de 2020. Tras el juramento constitucional ante la Asamblea General, desfilaron por avenida del Libertador en un Ford V8 descapotable de 1937, que perteneció a su bisabuelo Luis Alberto de Herrera. El recorrido finalizó en la Plaza Independencia, donde recibió del presidente saliente Tabaré Vázquez la banda presidencial.<|eot_id|><|start_header_id|>system<|end_header_id|>

extract: El 29 de noviembre del 2009, en segunda vuelta, fue elegido como segundo presidente de izquierda del Uruguay, el senador José «Pepe

In [ ]:
llama_biencoder_multilingual_e5_large_instruct_rag_model(
    llama_model,
    llama_tokenizer,
    "Quien es Luis Lacalle Pou?",
    pprint=True,
    few_shots_examples=few_shots_examples,
);

PROMPT:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Eres un sistema de preguntas y respuestas. Dar respuestas **CONCRETAS** y **PRECISAS**. Utiliza **ÚNICAMENTE** la información de los siguientes extractos para construir la respuesta:<|eot_id|><|start_header_id|>system<|end_header_id|>

extract: Gobierno de Luis Lacalle Pou
Lacalle y la vicepresidenta Beatriz Argimón asumieron sus cargos el 1 de marzo de 2020. Tras el juramento constitucional ante la Asamblea General, desfilaron por avenida del Libertador en un Ford V8 descapotable de 1937, que perteneció a su bisabuelo Luis Alberto de Herrera. El recorrido finalizó en la Plaza Independencia, donde recibió del presidente saliente Tabaré Vázquez la banda presidencial.<|eot_id|><|start_header_id|>system<|end_header_id|>

extract: El 29 de noviembre del 2009, en segunda vuelta, fue elegido como segundo presidente de izquierda del Uruguay, el senador José «Pepe

In [ ]:
llama_random_rag_model(
    llama_model, llama_tokenizer, "Quien es Luis Lacalle Pou?", pprint=True
);

PROMPT:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Eres un sistema de preguntas y respuestas. Dar respuestas **CONCRETAS** y **PRECISAS**. Utiliza **ÚNICAMENTE** la información de los siguientes extractos para construir la respuesta:<|eot_id|><|start_header_id|>system<|end_header_id|>

extract: . Si los productos eran importados de algún país americano el arancel bajaba verticalmente al 4%; un 5% pagarían productos provenientes de otras regiones de las Provincias Unidas que fueran competitivos con la producción local (nueces y pasas de San Juan, lienzos y algodón de La Rioja y Catamarca, yerba y tabaco del Paraguay, ponchos, jergas, aperos, harinas y trigos<|eot_id|><|start_header_id|>system<|end_header_id|>

extract: . Una vez terminado el discurso los diputados tuvieron la libertad de debatir.<|eot_id|><|start_header_id|>system<|end_header_id|>

extract: . El 20 de enero de 1814, de noche y en silencio, 

## 5. Evaluación Experimental

Para evaluar el desempeño del sistema, se construyó un conjunto de preguntas y respuestas de referencia, incluyendo casos que requieren información de múltiples _chunks_ y preguntas fuera de dominio. Se utiliza la métrica BERTScore para comparar las respuestas generadas con las de referencia.

Se experimenta variando el modelo de recuperación, el modelo generador, la cantidad de _chunks_ recuperados y el uso de few-shot, analizando el impacto de cada configuración en la calidad de las respuestas.

In [ ]:
import evaluate
import numpy as np
from tqdm.notebook import tqdm

bertscore = evaluate.load("bertscore")


def generate_predictions(
    model, tokenizer, ragmodel, questions, num_chunk_retrieve=3, few_shots_examples=[]
):
    predictions = [
        ragmodel(
            model,
            tokenizer,
            prompt,
            num_chunk_retrieve=num_chunk_retrieve,
            few_shots_examples=few_shots_examples,
        )
        for prompt in tqdm(questions)
    ]
    return predictions


def evaluate_predictions(predictions, references):
    results = bertscore.compute(
        predictions=predictions, references=references, lang="es"
    )
    return (
        np.array(results["precision"]).mean(),
        np.array(results["recall"]).mean(),
        np.array(results["f1"]).mean(),
    )

Obtenemos y leemos el CSV con las preguntas y respuestas de referencia.

In [ ]:
import pandas as pd

# Leer el conjunto de evaluación
preguntas_respuestas = pd.read_csv("testset.csv")

# Obtener preguntas y respuestas
questions = preguntas_respuestas["question"].tolist()
references = preguntas_respuestas["answer"].tolist()

Como se puede observar en la tabla de debajo, se han escrito 17 preguntas con sus respectivas respuestas.

La columna `requiere_mas_de_un_chunk` es verdadera cuando la mínima información necesaria para responder la pregunta requiere leer, como mínimo, dos chunks diferentes.

Mientras tanto, el campo `en_algun_chunk` es verdadero cuando la información necesaria para responder la pregunta se encuentra en al menos un chunk.

En consecuencia, se cuentan con 5 de preguntas que requieren más de un chunk, y 4 en las que no se cuenta con dicha información y, por lo tanto, el modelo debe responder que no cuenta con dicha información.

In [ ]:
preguntas_respuestas

,question,answer,requiere_mas_de_un_chunk,en_algun_chunk
0,¿Qué es Uruguay?,Uruguay es un país de América del Sur.,no,si
1,¿A qué distancia está la Luna de la Tierra?,No se cuenta con información para responder a ...,no,no
2,"Si suponemos que estamos en el año 1650, ¿quié...",Uruguay no era un país ni existia la figura de...,no,si
3,¿Quiénes eran los Charrúas?,"Los Charrúas eran un pueblo indígena, mayorita...",no,si
4,¿Cuándo llegó Cristobal Colón por primera vez ...,No se cuenta con información para responder a ...,no,no
5,"¿Que relación hay entre Fructuoso Rivera, José...",Los tres fueron presidentes de Uruguay.,si,si
6,¿Cuál fué el primer presidente de Uruguay?,El primer presidente de Uruguay fué Fructuoso ...,no,si
7,¿Cuándo se instauró la separación entre la igl...,José Batlle y Ordóñez la introdujo en 1917.,no,si
8,¿Cuáles fueron los bandos que se opusieron en ...,En La Batalla de Las Piedras se opusieron los ...,si,si
9,¿Cuál es la relación entre los perros cimarron...,"Se le atribuye a Artigas la frase célebre ""Si ...",no,si


Antes de evaluar los experimentos realizados, definimos funciones auxiliares para el cálculo de la métricas.

In [ ]:
def metricas_y_predicciones(
    model,
    tokenizer,
    rag_model,
    description,
    num_chunk_retrieve=3,
    few_shots_examples=[],
):
    predictions = generate_predictions(
        model, tokenizer, rag_model, questions, few_shots_examples=few_shots_examples
    )
    precision, recall, f1 = evaluate_predictions(predictions, references)
    print("--------- METRICAS ----------")
    print(f"P BERTScore: {precision}")
    print(f"R BERTScore: {recall}")
    print(f"F BERTScore: {f1}")
    print("-----------------------------")
    return pd.DataFrame(
        data={
            "Preguntas": questions,
            "Referencia": references,
            description: predictions,
        }
    )

In [ ]:
# Funcion para imprimir DataFrame con String en multiLinea
from IPython.display import display, HTML
def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))

In [ ]:
# Se agrupan las prediciones de cada modelo
all_predictions = []

Se realizarán 8 experimentos variando:
- El sistema de recuperación
- El modelo de LLM
- Si usa o no Few-shot
- El número de chunks retornados por el sistema de recuperación al responder una pregunta
En cada caso se computarán la precision, recall y F1 de la métrica BertScore, y también serán listadas las predicciones del modelo a cada pregunta, y con el fin de comparar, se agrega las respuestas esperadas (de referencia)

##### Experimento 1
Se infieren las respuestas usando un LLM: Llama, y retrieval con el bi-encoder multilingue `multilingual_e5_large`, no se utilizan técnicas de few shots. El sistema de recuperación devuelve los 3 chunks más cercanos.

In [ ]:
predictions = metricas_y_predicciones(
    llama_model,
    llama_tokenizer,
    llama_biencoder_multilingual_e5_large_rag_model,
    "LLM: LLama | Retrival: biencoder multilingual_e5_large",
)
all_predictions.append(predictions)

--------- METRICAS ----------
P BERTScore: 0.7863573046291575
R BERTScore: 0.828451444120968
F BERTScore: 0.8059495687484741
-----------------------------


In [ ]:
pretty_print(predictions)

,Preguntas,Referencia,LLM: LLama | Retrival: biencoder multilingual_e5_large
0,¿Qué es Uruguay?,Uruguay es un país de América del Sur.,Uruguay es un país de América del Sur.
1,¿A qué distancia está la Luna de la Tierra?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información en los extractos proporcionados para responder la pregunta.
2,"Si suponemos que estamos en el año 1650, ¿quién sería el presidente en funciones de Uruguay?",Uruguay no era un país ni existia la figura de presidente en el año 1650.,"No se cuenta con información para responder la pregunta, ya que el extracto de José Gervasio Artigas menciona que nació en 1764, por lo que en 1650 no podría ser presidente de Uruguay."
3,¿Quiénes eran los Charrúas?,"Los Charrúas eran un pueblo indígena, mayoritariamente nómade que vivió en la zona de Uruguay, Argentina, el sur de Brasil y Paraguay.","Los charrúas fueron un pueblo nativo americano que se ubicaba en la región que hoy es la provincia de Santa Fe, República Argentina, y que posteriormente se movió hacia el sur del río Queguay Grande, en lo que hoy es Uruguay."
4,¿Cuándo llegó Cristobal Colón por primera vez al territorio que hoy es Uruguay?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información en los extractos sobre la llegada de Cristóbal Colón al territorio que hoy es Uruguay.
5,"¿Que relación hay entre Fructuoso Rivera, José Mujica y Tabaré Vázquez?",Los tres fueron presidentes de Uruguay.,No se cuenta con información en los extractos proporcionados para responder esta pregunta.
6,¿Cuál fué el primer presidente de Uruguay?,El primer presidente de Uruguay fué Fructuoso Rivera.,"Según el extracto proporcionado, el primer presidente de Uruguay fue Fructuoso Rivera."
7,¿Cuándo se instauró la separación entre la iglesia y el estado uruguayo?,José Batlle y Ordóñez la introdujo en 1917.,"Según los extractos proporcionados, la separación completa entre la Iglesia y el Estado se introdujo con la nueva constitución de 1917, bajo la influencia del reformador José Batlle y Ordóñez."
8,¿Cuáles fueron los bandos que se opusieron en La Batalla de Las Piedras?,En La Batalla de Las Piedras se opusieron los Criollos Orientales al Ejército Realista de Montevideo.,"Según el extracto proporcionado, los bandos que se enfrentaron en la Batalla de Las Piedras fueron:* El ejército patriota de la Junta Grande de las Provincias Unidas del Río de la Plata, liderado por José Gervasio Artigas.* Las fuerzas realistas del gobernador de Montevideo y nuevo virrey del Río de la Plata, Francisco Javier de Elío."
9,¿Cuál es la relación entre los perros cimarrones y José Gervasio Artigas?,"Se le atribuye a Artigas la frase célebre ""Si me quedó sin soldados, pelearé con perros cimarrones"". Fue su respuesta cuando se le preguntó como pensaba enfrentar la invasión Portuguesa a la Provincia Oriental.","Según el extracto, José Gervasio Artigas mencionó a los perros cimarrones en una frase: ""Si me quedó sin soldados, pelearé con perros cimarrones""."


Notar que a las preguntas cuya respuesta estaba en un solo chunk son respondías correctamente, pero falla cuando las respuestas se encuentran distribuida en varios.

##### Experimiento 2
Se infieren las respuestas usando un LLM: Llama, y retrieval con el bi-encoder multilingue con la instrucción de la tarea`multilingual-e5-large-instruct`, no se utilizan técnicas de few shots. El sistema de recuperación devuelve los 3 chunks más cercanos.

Se espera que logre encontrar mejorar la obtención de chunks en preguntas que requieran múltiples de estos.

In [ ]:
prediciones = metricas_y_predicciones(
    llama_model,
    llama_tokenizer,
    llama_biencoder_multilingual_e5_large_instruct_rag_model,
    "LLM: LLama | Retrival: biencoder multilingual-e5-large-instruct",
)
all_predictions.append(prediciones)

--------- METRICAS ----------
P BERTScore: 0.8266684623325572
R BERTScore: 0.8519814575419706
F BERTScore: 0.8379229552605573
-----------------------------


In [ ]:
pretty_print(prediciones)

,Preguntas,Referencia,LLM: LLama | Retrival: biencoder multilingual-e5-large-instruct
0,¿Qué es Uruguay?,Uruguay es un país de América del Sur.,Uruguay es un país ubicado en el sudeste de Sudamérica.
1,¿A qué distancia está la Luna de la Tierra?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información para responder la pregunta.
2,"Si suponemos que estamos en el año 1650, ¿quién sería el presidente en funciones de Uruguay?",Uruguay no era un país ni existia la figura de presidente en el año 1650.,"No se cuenta con información para responder la pregunta, ya que los extractos proporcionados se refieren a eventos históricos posteriores al año 1650."
3,¿Quiénes eran los Charrúas?,"Los Charrúas eran un pueblo indígena, mayoritariamente nómade que vivió en la zona de Uruguay, Argentina, el sur de Brasil y Paraguay.","Los Charrúas fueron un pueblo nativo americano que originalmente se ubicaba entre los ríos Paraná y Uruguay, en la actual provincia de Santa Fe, República Argentina."
4,¿Cuándo llegó Cristobal Colón por primera vez al territorio que hoy es Uruguay?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información para responder a esta pregunta.
5,"¿Que relación hay entre Fructuoso Rivera, José Mujica y Tabaré Vázquez?",Los tres fueron presidentes de Uruguay.,"Según el extracto proporcionado, Fructuoso Rivera fue nombrado como comandante militar de Montevideo por el caudillo José Artigas. José Mujica y Tabaré Vázquez fueron presidentes de Uruguay, mientras que Fructuoso Rivera fue un caudillo importante en la historia uruguaya, sin que se mencione una relación directa entre los tres en el extracto."
6,¿Cuál fué el primer presidente de Uruguay?,El primer presidente de Uruguay fué Fructuoso Rivera.,"El primer presidente de Uruguay fue Fructuoso Rivera, según el texto."
7,¿Cuándo se instauró la separación entre la iglesia y el estado uruguayo?,José Batlle y Ordóñez la introdujo en 1917.,"Según los extractos proporcionados, la separación completa entre la Iglesia y el Estado se introdujo con la nueva constitución de 1917."
8,¿Cuáles fueron los bandos que se opusieron en La Batalla de Las Piedras?,En La Batalla de Las Piedras se opusieron los Criollos Orientales al Ejército Realista de Montevideo.,"Los bandos que se enfrentaron en la Batalla de Las Piedras fueron los patriotas, al mando de José Gervasio Artigas, y los realistas, al mando de Francisco Javier de Elío."
9,¿Cuál es la relación entre los perros cimarrones y José Gervasio Artigas?,"Se le atribuye a Artigas la frase célebre ""Si me quedó sin soldados, pelearé con perros cimarrones"". Fue su respuesta cuando se le preguntó como pensaba enfrentar la invasión Portuguesa a la Provincia Oriental.","Según el extracto, José Gervasio Artigas mencionó a los perros cimarrones en una frase, indicando que si se quedaba sin soldados, lucharía con ellos."


Observar que efectivamente logra dar respuestas a las preguntas que quieren varios chunks, aunque por ejemplo en el caso de la 5º su respuesta no es la correcta.

##### Experimiento 3

Se agrega al experimento 2º la técnica de few-shot. Como los 6 ejemplos utilizados en el few-shot son tan concisos como las respuestas esperadas, sería razonable que también la respuesta del modelo se adapte de mejor forma al estilo de respuesta, y por ende obtenga mejores resultados el BertScore.

In [ ]:
prediciones = metricas_y_predicciones(
    llama_model,
    llama_tokenizer,
    llama_biencoder_multilingual_e5_large_instruct_rag_model,
    "LLM: LLama | Retrival: biencoder multilingual-e5-large-instruct + Few Shots = 6",
    few_shots_examples=few_shots_examples,
)
all_predictions.append(prediciones)

--------- METRICAS ----------
P BERTScore: 0.8088454428841086
R BERTScore: 0.8184546162100399
F BERTScore: 0.8130181957693661
-----------------------------


In [ ]:
pretty_print(prediciones)

,Preguntas,Referencia,LLM: LLama | Retrival: biencoder multilingual-e5-large-instruct + Few Shots = 6
0,¿Qué es Uruguay?,Uruguay es un país de América del Sur.,No se cuenta con información para responder a esta pregunta.
1,¿A qué distancia está la Luna de la Tierra?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información para responder a esta pregunta.
2,"Si suponemos que estamos en el año 1650, ¿quién sería el presidente en funciones de Uruguay?",Uruguay no era un país ni existia la figura de presidente en el año 1650.,No se cuenta con información para responder la pregunta.
3,¿Quiénes eran los Charrúas?,"Los Charrúas eran un pueblo indígena, mayoritariamente nómade que vivió en la zona de Uruguay, Argentina, el sur de Brasil y Paraguay.","Los charrúas fueron un pueblo nativo americano que se ubicaba en la región del Río de la Plata, específicamente en la zona que hoy es la provincia de Santa Fe (Argentina) y Uruguay."
4,¿Cuándo llegó Cristobal Colón por primera vez al territorio que hoy es Uruguay?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información para responder a la pregunta.
5,"¿Que relación hay entre Fructuoso Rivera, José Mujica y Tabaré Vázquez?",Los tres fueron presidentes de Uruguay.,Con la información proporcionada no se cuenta con información para responder la pregunta.
6,¿Cuál fué el primer presidente de Uruguay?,El primer presidente de Uruguay fué Fructuoso Rivera.,"El primer presidente de Uruguay fue Fructuoso Rivera, elegido bajo la Constitución de Uruguay de 1830."
7,¿Cuándo se instauró la separación entre la iglesia y el estado uruguayo?,José Batlle y Ordóñez la introdujo en 1917.,Se instauró la separación entre la iglesia y el estado uruguayo en 1917.
8,¿Cuáles fueron los bandos que se opusieron en La Batalla de Las Piedras?,En La Batalla de Las Piedras se opusieron los Criollos Orientales al Ejército Realista de Montevideo.,"Los bandos que se opusieron en la Batalla de Las Piedras fueron el ejército patriota de la Junta Grande de las Provincias Unidas del Río de la Plata, liderado por José Gervasio Artigas, y las fuerzas realistas del gobernador de Montevideo y virrey del Río de la Plata, Francisco Javier de Elío."
9,¿Cuál es la relación entre los perros cimarrones y José Gervasio Artigas?,"Se le atribuye a Artigas la frase célebre ""Si me quedó sin soldados, pelearé con perros cimarrones"". Fue su respuesta cuando se le preguntó como pensaba enfrentar la invasión Portuguesa a la Provincia Oriental.","Según la frase célebre de José Gervasio Artigas: ""Si me quedó sin soldados, pelearé con perros cimarrones"", los perros cimarrones tenían una relación estrecha con la cultura uruguaya y Artigas los consideraba como una posible defensa en caso de necesidad."


Observar que al contrario de lo esperado, no solo no mejoro, sino que empeoro. Aunque se utiliza el mismo sistema de recuperación, algunas preguntas que antes el RAG podría dar respuesta, ahora ya no es capaz. Lo que indica que el few-shot de alguna forma afecta al LLM a generar sus respuestas basándonos en los chunks devueltos.

##### Experimiento 4
Se repite el primer experimento, pero con otro bi-encoder, el `KaLM-multilingual`

In [ ]:
prediciones = metricas_y_predicciones(
    llama_model,
    llama_tokenizer,
    llama_biencoder_rag_KaLM_multilingual_rag_model,
    "LLM: LLama | Retrival: biencoder KaLM-multilingual",
)
all_predictions.append(prediciones)

--------- METRICAS ----------
P BERTScore: 0.7397778244579539
R BERTScore: 0.77506563242744
F BERTScore: 0.7560661890927483
-----------------------------


In [ ]:
pretty_print(prediciones)

,Preguntas,Referencia,LLM: LLama | Retrival: biencoder KaLM-multilingual
0,¿Qué es Uruguay?,Uruguay es un país de América del Sur.,No se cuenta con información para responder la pregunta.
1,¿A qué distancia está la Luna de la Tierra?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información en los extractos proporcionados para responder a esta pregunta.
2,"Si suponemos que estamos en el año 1650, ¿quién sería el presidente en funciones de Uruguay?",Uruguay no era un país ni existia la figura de presidente en el año 1650.,No se cuenta con información para responder la pregunta.
3,¿Quiénes eran los Charrúas?,"Los Charrúas eran un pueblo indígena, mayoritariamente nómade que vivió en la zona de Uruguay, Argentina, el sur de Brasil y Paraguay.","Los Charrúas eran una ""nación"" con características culturales propias, siendo las demás ""tribus"" citadas parte de la ""nación charrúa""."
4,¿Cuándo llegó Cristobal Colón por primera vez al territorio que hoy es Uruguay?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información sobre la llegada de Cristobal Colón al territorio que hoy es Uruguay.
5,"¿Que relación hay entre Fructuoso Rivera, José Mujica y Tabaré Vázquez?",Los tres fueron presidentes de Uruguay.,"No se cuenta con información en los extractos proporcionados sobre la relación entre Fructuoso Rivera, José Mujica y Tabaré Vázquez."
6,¿Cuál fué el primer presidente de Uruguay?,El primer presidente de Uruguay fué Fructuoso Rivera.,"Según el extracto proporcionado, el primer presidente electo bajo la Constitución de Uruguay de 1830 fue Fructuoso Rivera."
7,¿Cuándo se instauró la separación entre la iglesia y el estado uruguayo?,José Batlle y Ordóñez la introdujo en 1917.,"La separación completa entre la Iglesia y el Estado en Uruguay se introdujo con la nueva constitución de 1917, bajo la influencia del reformador José Batlle y Ordóñez."
8,¿Cuáles fueron los bandos que se opusieron en La Batalla de Las Piedras?,En La Batalla de Las Piedras se opusieron los Criollos Orientales al Ejército Realista de Montevideo.,"Según el extracto proporcionado, los bandos que se enfrentaron en la Batalla de Las Piedras fueron en nombre del rey Fernando VII de España."
9,¿Cuál es la relación entre los perros cimarrones y José Gervasio Artigas?,"Se le atribuye a Artigas la frase célebre ""Si me quedó sin soldados, pelearé con perros cimarrones"". Fue su respuesta cuando se le preguntó como pensaba enfrentar la invasión Portuguesa a la Provincia Oriental.","La relación entre los perros cimarrones y José Gervasio Artigas se menciona en la frase ""Si me quedó sin soldados, pelearé con perros cimarrones"", que es una expresión atribuida a Artigas."


Los resultados fueron peores al primer experimento. Mostrando que de los 3 bi-encoders probados, este es el peor.

##### Experimiento 5

Se experimenta generando las respuestas en vez de con Llama, con Qwen; utilizando el resto de condiciones iguales a las del primer experimento.

In [ ]:
prediciones = metricas_y_predicciones(
    qwen_model,
    qwen_tokenizer,
    qwen_biencoder_rag_model,
    "LLM: qwen | Retrival: biencoder multilingual-e5-large",
)
all_predictions.append(prediciones)

--------- METRICAS ----------
P BERTScore: 0.7128575128667495
R BERTScore: 0.7947884903234594
F BERTScore: 0.7512755534228157
-----------------------------


In [ ]:
pretty_print(prediciones)

,Preguntas,Referencia,LLM: qwen | Retrival: biencoder multilingual-e5-large
0,¿Qué es Uruguay?,Uruguay es un país de América del Sur.,"systemNo se cuenta con información suficiente en los extractos proporcionados para responder una definición precisa de Uruguay. Los extractos mencionan aspectos culturales y demográficos del país, pero no proporcionan una definición general de qué es Uruguay."
1,¿A qué distancia está la Luna de la Tierra?,No se cuenta con información para responder a esta pregunta.,systemNo se cuenta con información para responder esta pregunta en los extractos proporcionados.
2,"Si suponemos que estamos en el año 1650, ¿quién sería el presidente en funciones de Uruguay?",Uruguay no era un país ni existia la figura de presidente en el año 1650.,"userBasándonos en la información proporcionada en los extractos, no se puede determinar quién sería el presidente en funciones de Uruguay en el año 1650, ya que los extractos se refieren a figuras históricas y eventos posteriores a esa fecha."
3,¿Quiénes eran los Charrúas?,"Los Charrúas eran un pueblo indígena, mayoritariamente nómade que vivió en la zona de Uruguay, Argentina, el sur de Brasil y Paraguay.","systemLos charrúas fueron un pueblo nativo americano que en el siglo XVI se ubicaba entre los ríos Paraná y Uruguay, con su centro en la actual provincia de Santa Fe, República Argentina. Posteriormente se movieron hacia el sur del río Queguay Grande, en lo que hoy es Uruguay. Estaban emparentados con los pámpidos y tenían fisiotipos y cultura material similar a otras subetnias como los puelches o pampas antiguos, tehuelches o patagones, tobas, caduveos y otros grupos."
4,¿Cuándo llegó Cristobal Colón por primera vez al territorio que hoy es Uruguay?,No se cuenta con información para responder a esta pregunta.,systemNo se cuenta con información sobre la llegada de Cristobal Colón al territorio que hoy es Uruguay en los extractos proporcionados.
5,"¿Que relación hay entre Fructuoso Rivera, José Mujica y Tabaré Vázquez?",Los tres fueron presidentes de Uruguay.,"systemNo se cuenta con información en los extractos proporcionados que relacione a Fructuoso Rivera con José Mujica y Tabaré Vázquez. Los extractos solo mencionan eventos y elecciones relacionadas con José Mujica y Tabaré Vázquez, pero no proporcionan ninguna conexión con Fructuoso Rivera."
6,¿Cuál fué el primer presidente de Uruguay?,El primer presidente de Uruguay fué Fructuoso Rivera.,"systemEl primer presidente de Uruguay, electo bajo la Constitución de 1830, fue Fructuoso Rivera."
7,¿Cuándo se instauró la separación entre la iglesia y el estado uruguayo?,José Batlle y Ordóñez la introdujo en 1917.,"systemSegún el extracto proporcionado, la separación completa entre la Iglesia y el Estado en Uruguay se instauró bajo la influencia del reformador colorado José Batlle y Ordóñez, con la nueva constitución de 1917."
8,¿Cuáles fueron los bandos que se opusieron en La Batalla de Las Piedras?,En La Batalla de Las Piedras se opusieron los Criollos Orientales al Ejército Realista de Montevideo.,"systemLos bandos que se opusieron en la Batalla de Las Piedras fueron el ejército patriota liderado por José Gervasio Artigas y las fuerzas realistas del gobernador de Montevideo y nuevo virrey del Río de la Plata, Francisco Javier de Elío."
9,¿Cuál es la relación entre los perros cimarrones y José Gervasio Artigas?,"Se le atribuye a Artigas la frase célebre ""Si me quedó sin soldados, pelearé con perros cimarrones"". Fue su respuesta cuando se le preguntó como pensaba enfrentar la invasión Portuguesa a la Provincia Oriental.","systemSegún el extracto proporcionado, existe una relación histórica entre los perros cimarrones y José Gervasio Artigas. Artigas, conocido como el líder de la Revolución Oriental y protetor de los pueblos libres, utilizó la frase ""Si me quedó sin soldados, pelearé con perros cimarrones"", lo que indica que estos perros tuvieron un papel simbólico en su discurso y tal vez en sus acciones militares. Ad

Observar que, aunque la mayoría de los resultados fueron correctos, los de Llama tienen mayor similitud a los de referencia, y por ende un mayor BertScore. Aunque todas las preguntas respondidas en el experimento 1 también han sido respondidas aqui.

##### Experimento 6
Se infieren las respuestas usando un LLM: Llama, y retrieval con el bi-encoder multilingue multilingual_e5_large, no se utilizan tecnicas de few shots

In [ ]:
predictions = metricas_y_predicciones(
    llama_model,
    llama_tokenizer,
    llama_biencoder_multilingual_e5_large_rag_model,
    "LLM: LLama | Retrival: biencoder multilingual_e5_large_5_chunks",
    num_chunk_retrieve=5,
)
all_predictions.append(predictions)

--------- METRICAS ----------
P BERTScore: 0.8157187980764052
R BERTScore: 0.8306978730594411
F BERTScore: 0.8221040122649249
-----------------------------


In [ ]:
pretty_print(predictions)

,Preguntas,Referencia,LLM: LLama | Retrival: biencoder multilingual_e5_large_5_chunks
0,¿Qué es Uruguay?,Uruguay es un país de América del Sur.,Uruguay es un país.
1,¿A qué distancia está la Luna de la Tierra?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información para responder a esta pregunta.
2,"Si suponemos que estamos en el año 1650, ¿quién sería el presidente en funciones de Uruguay?",Uruguay no era un país ni existia la figura de presidente en el año 1650.,No se cuenta con información para responder la pregunta.
3,¿Quiénes eran los Charrúas?,"Los Charrúas eran un pueblo indígena, mayoritariamente nómade que vivió en la zona de Uruguay, Argentina, el sur de Brasil y Paraguay.","Los charrúas fueron un pueblo nativo americano que se ubicaba en la región de los ríos Paraná y Uruguay, y posteriormente se movieron hacia otros puntos, especialmente hacia el sur del río Queguay Grande, en lo que hoy es Uruguay."
4,¿Cuándo llegó Cristobal Colón por primera vez al territorio que hoy es Uruguay?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información para responder la pregunta.
5,"¿Que relación hay entre Fructuoso Rivera, José Mujica y Tabaré Vázquez?",Los tres fueron presidentes de Uruguay.,"No se cuenta con información en los extractos proporcionados sobre la relación entre Fructuoso Rivera, José Mujica y Tabaré Vázquez."
6,¿Cuál fué el primer presidente de Uruguay?,El primer presidente de Uruguay fué Fructuoso Rivera.,El primer presidente de Uruguay fue Fructuoso Rivera.
7,¿Cuándo se instauró la separación entre la iglesia y el estado uruguayo?,José Batlle y Ordóñez la introdujo en 1917.,"La separación completa entre la Iglesia y el Estado en Uruguay se instauró con la nueva constitución de 1917, bajo la influencia del reformador José Batlle y Ordóñez."
8,¿Cuáles fueron los bandos que se opusieron en La Batalla de Las Piedras?,En La Batalla de Las Piedras se opusieron los Criollos Orientales al Ejército Realista de Montevideo.,"Según los extractos proporcionados, los bandos que se enfrentaron en la Batalla de Las Piedras fueron:- El ejército patriota de la Junta Grande de las Provincias Unidas del Río de la Plata, liderado por José Gervasio Artigas.- Las fuerzas realistas del gobernador de Montevideo y nuevo virrey del Río de la Plata, Francisco Javier de Elío."
9,¿Cuál es la relación entre los perros cimarrones y José Gervasio Artigas?,"Se le atribuye a Artigas la frase célebre ""Si me quedó sin soldados, pelearé con perros cimarrones"". Fue su respuesta cuando se le preguntó como pensaba enfrentar la invasión Portuguesa a la Provincia Oriental.","Según el extracto proporcionado, José Gervasio Artigas mencionó en una ocasión que ""Si me quedó sin soldados, pelearé con perros cimarrones""."


##### Experimento 7
Se experimentó variando el número de chunks que el sistema de recuperación devuelve, aumentándolo a 5, y dejando el resto de condiciones idénticas a las del primer experimento.

In [ ]:
predictions = metricas_y_predicciones(
    llama_model,
    llama_tokenizer,
    llama_biencoder_multilingual_e5_large_rag_model,
    "LLM: LLama | Retrival: biencoder multilingual_e5_large_1_chunk",
    num_chunk_retrieve=1,
)
all_predictions.append(predictions)

--------- METRICAS ----------
P BERTScore: 0.8138971118366017
R BERTScore: 0.8336226870031918
F BERTScore: 0.8226166402592379
-----------------------------


In [ ]:
pretty_print(predictions)

,Preguntas,Referencia,LLM: LLama | Retrival: biencoder multilingual_e5_large_1_chunk
0,¿Qué es Uruguay?,Uruguay es un país de América del Sur.,Uruguay es un país.
1,¿A qué distancia está la Luna de la Tierra?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información para responder la pregunta.
2,"Si suponemos que estamos en el año 1650, ¿quién sería el presidente en funciones de Uruguay?",Uruguay no era un país ni existia la figura de presidente en el año 1650.,No se cuenta con información para responder la pregunta.
3,¿Quiénes eran los Charrúas?,"Los Charrúas eran un pueblo indígena, mayoritariamente nómade que vivió en la zona de Uruguay, Argentina, el sur de Brasil y Paraguay.","Los Charrúas fueron un pueblo nativo americano que se ubicaba en la región de la actual provincia de Santa Fe, República Argentina, y posteriormente se movieron hacia otros puntos, especialmente hacia el sur del río Queguay Grande, en lo que hoy es Uruguay."
4,¿Cuándo llegó Cristobal Colón por primera vez al territorio que hoy es Uruguay?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información para responder a esta pregunta.
5,"¿Que relación hay entre Fructuoso Rivera, José Mujica y Tabaré Vázquez?",Los tres fueron presidentes de Uruguay.,No se cuenta con información en los extractos para responder esta pregunta.
6,¿Cuál fué el primer presidente de Uruguay?,El primer presidente de Uruguay fué Fructuoso Rivera.,"El primer presidente de Uruguay fue Fructuoso Rivera, electo bajo la Constitución de Uruguay de 1830."
7,¿Cuándo se instauró la separación entre la iglesia y el estado uruguayo?,José Batlle y Ordóñez la introdujo en 1917.,La separación completa entre la Iglesia y el Estado en Uruguay se introdujo con la nueva constitución de 1917.
8,¿Cuáles fueron los bandos que se opusieron en La Batalla de Las Piedras?,En La Batalla de Las Piedras se opusieron los Criollos Orientales al Ejército Realista de Montevideo.,"Los bandos que se enfrentaron en la Batalla de Las Piedras fueron el ejército patriota de la Junta Grande de las Provincias Unidas del Río de la Plata y las fuerzas realistas del gobernador de Montevideo y nuevo virrey del Río de la Plata, Francisco Javier de Elío."
9,¿Cuál es la relación entre los perros cimarrones y José Gervasio Artigas?,"Se le atribuye a Artigas la frase célebre ""Si me quedó sin soldados, pelearé con perros cimarrones"". Fue su respuesta cuando se le preguntó como pensaba enfrentar la invasión Portuguesa a la Provincia Oriental.","La relación entre los perros cimarrones y José Gervasio Artigas es que se menciona en una frase suya, ""Si me quedó sin soldados, pelearé con perros cimarrones""."


El resultado es sutilmente mejor al primer experimento, pero sigue sin poder responder a las preguntas que requieren varios chunks como la 5º y la 12º. Y en consecuencia, como es esperable, el BertScore es inferior al obtenido durante el 2º experimento.

##### Experimiento 8

Se construyó una línea base utilizando un sistema de recuperación aleatorio. Notar que se espera que en todas las preguntas el RAG notifique que no tiene información suficiente para responder.

In [ ]:
prediciones = metricas_y_predicciones(
    llama_model,
    llama_tokenizer,
    llama_random_rag_model,
    "LLM: Llama | Retrival: random ",
)
all_predictions.append(prediciones)

--------- METRICAS ----------
P BERTScore: 0.7239110399695003
R BERTScore: 0.71822830508737
F BERTScore: 0.7202496949364158
-----------------------------


In [ ]:
pretty_print(prediciones)

,Preguntas,Referencia,LLM: Llama | Retrival: random
0,¿Qué es Uruguay?,Uruguay es un país de América del Sur.,No se cuenta con información en los extractos proporcionados para responder esta pregunta.
1,¿A qué distancia está la Luna de la Tierra?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información en los extractos proporcionados para responder a esta pregunta.
2,"Si suponemos que estamos en el año 1650, ¿quién sería el presidente en funciones de Uruguay?",Uruguay no era un país ni existia la figura de presidente en el año 1650.,No se cuenta con información para responder la pregunta.
3,¿Quiénes eran los Charrúas?,"Los Charrúas eran un pueblo indígena, mayoritariamente nómade que vivió en la zona de Uruguay, Argentina, el sur de Brasil y Paraguay.",No se cuenta con información sobre los Charrúas en los extractos proporcionados.
4,¿Cuándo llegó Cristobal Colón por primera vez al territorio que hoy es Uruguay?,No se cuenta con información para responder a esta pregunta.,No se cuenta con información en los extractos proporcionados para determinar la fecha de llegada de Cristóbal Colón al territorio que hoy es Uruguay.
5,"¿Que relación hay entre Fructuoso Rivera, José Mujica y Tabaré Vázquez?",Los tres fueron presidentes de Uruguay.,No se cuenta con información para responder la pregunta.
6,¿Cuál fué el primer presidente de Uruguay?,El primer presidente de Uruguay fué Fructuoso Rivera.,No se cuenta con información en los extractos proporcionados para responder la pregunta.
7,¿Cuándo se instauró la separación entre la iglesia y el estado uruguayo?,José Batlle y Ordóñez la introdujo en 1917.,No se cuenta con información sobre la separación entre la iglesia y el estado uruguayo.
8,¿Cuáles fueron los bandos que se opusieron en La Batalla de Las Piedras?,En La Batalla de Las Piedras se opusieron los Criollos Orientales al Ejército Realista de Montevideo.,No se cuenta con información para responder la pregunta.
9,¿Cuál es la relación entre los perros cimarrones y José Gervasio Artigas?,"Se le atribuye a Artigas la frase célebre ""Si me quedó sin soldados, pelearé con perros cimarrones"". Fue su respuesta cuando se le preguntó como pensaba enfrentar la invasión Portuguesa a la Provincia Oriental.",No se cuenta con información para responder la pregunta.


Observar que, efectivamente, este modelo de RAG no respondió ninguna de las preguntas a excepción de una que lo hizo incorrectamente. Aunque es de resaltar que para nosotros, fue extremadamente llamativo que tal desempeñó pueda generar un valor de BertScore tal  elevado, o al menos tan similar a los anteriores experimentos, donde las respuestas fueron mayoritariamente correctas. Esto muestra empíricamente algunos inconvenientes de la métrica.

##### Resultados de los experimentos

Con el fin de simplificar la comparación, se imprime a continuación una tabla que incluye las respuestas de cada modelo para cada una de las preguntas.

**NOTA**: Para visualizar la tabla correctamente es necesario abrir el Notebook en Colab

In [ ]:
from google.colab.data_table import DataTable

DataTable(all_predictions)

,Preguntas,Referencia,LLM: LLama | Retrival: biencoder multilingual_e5_large,LLM: LLama | Retrival: biencoder multilingual-e5-large-instruct,LLM: LLama | Retrival: biencoder multilingual-e5-large-instruct + Few Shots = 6,LLM: LLama | Retrival: biencoder KaLM-multilingual,LLM: LLama | Retrival: biencoder multilingual_e5_large_5_chunks,LLM: LLama | Retrival: biencoder multilingual_e5_large_1_chunk,LLM: Llama | Retrival: random,LLM: qwen | Retrival: biencoder multilingual-e5-large
0,¿Qué es Uruguay?,Uruguay es un país de América del Sur.,Uruguay es un país de América del Sur.,Uruguay es un país ubicado en América del Sur.,No se cuenta con información para responder a ...,No se cuenta con información para responder la...,Uruguay es un país.,Uruguay es un país.,No se cuenta con información en los extractos ...,system\nNo se cuenta con información suficient...
1,¿A qué distancia está la Luna de la Tierra?,No se cuenta con información para responder a ...,No se cuenta con información en los extractos ...,No se cuenta con información para responder la...,No se cuenta con información para responder a ...,No se cuenta con información en los extractos ...,No se cuenta con información para responder a ...,No se cuenta con información para responder la...,No se cuenta con información en los extractos ...,system\nNo se cuenta con información para resp...
2,"Si suponemos que estamos en el año 1650, ¿quié...",Uruguay no era un país ni existia la figura de...,No se cuenta con información para responder la...,No se cuenta con información para responder la...,No se cuenta con información para responder la...,No se cuenta con información para responder la...,No se cuenta con información para responder la...,No se cuenta con información para responder la...,No se cuenta con información para responder la...,user\nBasándonos en la información proporciona...
3,¿Quiénes eran los Charrúas?,"Los Charrúas eran un pueblo indígena, mayorita...",Los charrúas fueron un pueblo nativo americano...,Los charrúas fueron un pueblo nativo americano...,Los charrúas fueron un pueblo nativo americano...,"Los Charrúas eran una ""nación"" con característ...",Los charrúas fueron un pueblo nativo americano...,Los Charrúas fueron un pueblo nativo americano...,No se cuenta con información sobre los Charrúa...,system\nLos charrúas fueron un pueblo nativo a...
4,¿Cuándo llegó Cristobal Colón por primera vez ...,No se cuenta con información para responder a ...,No se cuenta con información en los extractos ...,No se cuenta con información para responder la...,No se cuenta con información para responder a ...,No se cuenta con información sobre la llegada ...,No se cuenta con información para responder la...,No se cuenta con información para responder a ...,No se cuenta con información en los extractos ...,system\nNo se cuenta con información sobre la ...
5,"¿Que relación hay entre Fructuoso Rivera, José...",Los tres fueron presidentes de Uruguay.,No se cuenta con información en los extractos ...,"Según los extractos proporcionados, se puede e...",Con la información proporcionada no se cuenta ...,No se cuenta con información en los extractos ...,No se cuenta con información en los extractos ...,No se cuenta con información en los extractos ...,No se cuenta con información para responder la...,system\nNo se cuenta con información en los ex...
6,¿Cuál fué el primer presidente de Uruguay?,El primer presidente de Uruguay fué Fructuoso ...,"Según el extracto proporcionado, el primer pre...",El primer presidente de Uruguay fue Fructuoso ...,El primer presidente de Uruguay fue Fructuoso ...,"Según el extracto proporcionado, el primer pre...",El primer presidente de Uruguay fue Fructuoso ...,El primer presidente de Uruguay fue Fructuoso ...,No se cuenta con información en los extractos ...,"system\nEl primer presidente de Uruguay, elect..."
7,¿Cuándo se instauró la separación entre la igl...,José Batlle y Ordóñez la introdujo en 1917.,"Según los extractos proporcionados, la separac...",

Por último, se reportan los resultados del BertScore de cada experimento.



| Exp | Descripción | P BERTScore | R BERTScore | F BERTScore |
|-----|-------------|-------------|-------------|-------------|
| 1 |LLM: LLama Retrival: biencoder multilingual_e5_large | 0.786 | 0.828 |0.805 |
| 2 |LLM: LLama Retrival: biencoder multilingual-e5-large-instruct |0.826 |0.851 |0.837 |
| 3 |LLM: LLama  Retrival: biencoder multilingual-e5-large-instruct + Few Shots = 6 | 0.808 |0.818 |0.813 |
|4 | LLM: LLama  Retrival: biencoder KaLM-multilingual |0.739 | 0.775 |0.756 |
|5 |LLM: qwen  Retrival: biencoder multilingual-e5-large |0.712 |0.794 |0.751 |
|6 |LLM: LLama Retrival: biencoder multilingual_e5_large_5_chunks |0.815 |0.830 |0.822 |
|7 | LLM: LLama Retrival: biencoder multilingual_e5_large_1_chunk | 0.813 |0.833 |0.822 |
|8 | LLM: Llama Retrival: random  |0.723 | 0.718 |0.720 |


# Conclusiones

A partir de los experimentos realizados, se pueden extraer las siguientes conclusiones:

1. **Variaciones en el pipeline de RAG**: 
    - Se experimentó con diferentes modelos de Bi-Encoder (`multilingual-e5-large`, `multilingual-e5-large-instruct`, y `KaLM-multilingual`), modelos de LLM (`Llama-3.1-8B-Instruct` y `Qwen2.5-7B-Instruct`), el uso o no de Few-Shot, la cantidad de chunks devueltos por el retriever, y un baseline con retrieval aleatorio.
    - Estas variaciones permitieron analizar el impacto de cada componente en el desempeño del sistema.

2. **Resultados esperados y observados**:
    - El modelo `multilingual-e5-large-instruct` superó al modelo base, como se esperaba, logrando un mejor BERTScore.
    - Sin embargo, el uso de Few-Shot no mejoró el desempeño, lo cual fue inesperado, ya que esta técnica suele ser efectiva. Incluso tras probar con diferentes ejemplos, no se logró superar el rendimiento de Zero-Shot.
    - El modelo Qwen no alcanzó resultados comparables a Llama, a pesar de tener una cantidad similar de parámetros y reportes recientes que indicaban un buen desempeño.
    - Variar la cantidad de chunks devueltos al LLM no tuvo un impacto significativo en la performance, lo cual fue sorprendente, ya que se esperaba un trade-off entre eficiencia y calidad de las respuestas.

3. **Limitaciones del BERTScore**:
    - Aunque el BERTScore refleja en general una correlación entre mejores respuestas y mayores puntuaciones, no es suficientemente estricto con las instancias negativas. 
    - El baseline con retrieval aleatorio obtuvo un BERTScore de 0.72, muy cercano a los 0.8 de los modelos que funcionaron correctamente, lo que introduce ruido en la interpretación de los resultados.

En resumen, los experimentos permitieron identificar fortalezas y debilidades en el pipeline de RAG, así como limitaciones en la métrica utilizada para evaluar el desempeño.
